In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np

from glob import glob
from tqdm import notebook

from model import get_model
from simple_loader import get_data
from preprocessing import load_scaler
from config import ModelConfig

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080, compute capability 8.6


In [2]:
def least_square(x, y):
    from sklearn.linear_model import LinearRegression
    LR = LinearRegression()
    LR.fit(x.reshape(-1, 1), y.reshape(-1, 1))
    
    return LR.coef_[0], LR.intercept_ # return w, b in y = wx + b

In [3]:
def calculate_ci(y, z=1.96):
    n = int(y.shape[0])
    n_sqrt, mean, std = np.sqrt(n), y_val[-1, :], np.std(y, axis=0)
#     print(y)
#     print(n_sqrt)
#     print(mean)
#     print(std)
    upper_ci = mean + z * std / n_sqrt
    lower_ci = mean - z * std / n_sqrt
    return upper_ci, lower_ci

In [4]:
def get_predicted_data(model, idx):
    scalers = load_scaler(idx, 'scalers')
    scaler = scalers['Close']
        
    data = get_data(
        n_day=ModelConfig.n_day, n_future=ModelConfig.n_future,
        data_path=data_path, drop=['Date']
    )
    x_data, y_true = data
        
    if x_data.size == 0 or x_data.shape[0] < ModelConfig.n_future:
        print(f'{idx} passed.')
        return None
       
    y_pred = model.predict(x_data)

    y_true = scaler.inverse_transform(y_true)
    y_pred = scaler.inverse_transform(y_pred)
    
    return x_data, y_true, y_pred

In [5]:
model = get_model(input_shape=None)

found best model: ./models\AcademyProject-epoch-011+val_loss-0.0047161025.hdf5


In [6]:
data_dir = './revised_dataset/scaled_csv/'
data_list = glob(data_dir + '*.csv')

img_path_format = './scaled_32-series_model_forecasting_images/prediction_{idx}.png'
scaler_format = './revised_dataset/scaler/{idx:06d}_scaler.sclr'

In [7]:
prog_bar = notebook.tqdm(data_list)
for data_path in prog_bar:
    idx = data_path.replace('.csv', '').split('_')[-1]
    prog_bar.set_description_str(f'code: {idx}')
        
    img_path = img_path_format.format(idx=idx)
    
    if os.path.exists(img_path):
        continue
    
    data = get_predicted_data(model, idx)
    if data is None:
        print(f'{idx} passed.')
        continue
        
    x_data, y_true, y_pred = data
    
#     val_length = int((1 - ModelConfig.val_split) * x_data.shape[0])
#     y_true = y_true[:val_length+n_future, 0].squeeze()
#     if val_length+n_future != y_true.shape[0]:
#         print(f'{idx} not enough data.')
#         continue
    
    # shape comparision
    # y_true.shape = (val_length+n_future, )
    # y_fit.shape  = (val_length, ModelConfig.n_future)
    # y_val.shape  = (n_future, n_future)
    n_future = ModelConfig.n_future // 2
    
    # last 20%(val_split) data is validation data

    # flatten y_true, y_fit, y_val data
    val_length = x_data.shape[0]
    y_true = y_true[:val_length+n_future, 0].squeeze()
    
    # use day-1 fit only
    y_fit = y_pred[:val_length, :]
    
    y_val = np.zeros((n_future, n_future))
    for i, idx in enumerate(range(val_length-n_future, val_length)):
        y_temp = y_fit[idx, n_future-(i+1):]
        
        y_val[i, :] = y_temp[:n_future]
    
    x_total = np.arange(y_true.shape[0])
    x_val = np.arange(val_length)
    x_future = np.arange(val_length, val_length+n_future)
    w, b = least_square(x_future, y_val[-1, :])  # using only the lastest prediction
    y_lr = w * x_future + b
    
    upper_ci, lower_ci = calculate_ci(y_val)
    
    # save figure    
    fig = plt.figure(figsize=(20, 16))
    
    plt.plot(x_total, y_true, color='k', label='y_true')
    plt.plot(x_val, y_fit[:, 0], label='y_fit')
    plt.plot(x_future, y_lr, color='g', label='y_trends')
    
    plt.fill_between(x_future, upper_ci, lower_ci, alpha=0.3)
    
    plt.title(f'Deeplearning code {idx} zoomed-in')
    plt.xlabel('Date')
    plt.ylabel('price')

    plt.legend()
    plt.savefig(img_path)
    plt.close()
    
    # zoomed-in figure
    fig = plt.figure(figsize=(20, 16))
    plt.plot(x_total[-50-n_future:], y_true[-50-n_future:], color='k', label='y_true')
    plt.plot(x_val[-50:], y_fit[-50:, 0], label='y_fit')
    plt.plot(x_future, y_lr, color='g', label='y_trends')
    
    plt.fill_between(x_future, upper_ci, lower_ci, alpha=0.3)
    
    plt.title(f'Deeplearning code {idx} zoomed-in')
    plt.xlabel('Date')
    plt.ylabel('price')

    plt.legend()
    plt.savefig(img_path.replace('.png', '_zoomed-in.png'))
    plt.close()

089860 passed.
089860 passed.


099390 passed.
099390 passed.


099430 passed.
099430 passed.


114840 passed.
114840 passed.


139990 passed.
139990 passed.


...

400560 passed.
400560 passed.


400760 passed.
400760 passed.


400840 passed.
400840 passed.


402340 passed.
402340 passed.


404990 passed.
404990 passed.
